In [8]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import iris
import numpy as np
import sys
sys.path.append('/pf/b/b381639/Analysis')
import fxtian_function as fx
from iris.time import PartialDateTime
import datetime
import cf_units
import iris.coord_categorisation
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np
import xarray as ar

# %% list of data to read in
def data_dict(myid):
    if myid == 'sst':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/betzy_Lcam6_1.cam.h1.0010-01-01-00000.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'SST'))
        }
    if myid == 'DT':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/DT_150_tropical_ocean.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'SST'))
        }
    if myid == 'DQ':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/DQ_daily_LCAM6.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'DQ'))
        }
        
    if myid == 'wsp10':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/betzy_Lcam6_1.cam.h1.0010-01-01-00000.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'U10'))
        }
    elif myid == 'slhf':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/betzy_Lcam6_1.cam.h1.0010-01-01-00000.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'LHFLX'))
        }

    elif myid == 'sshf':
        mydict = {
            'indir': '/work/bb1153/b381639/NorESM',
            'infile': '/work/bb1153/b381639/NorESM/betzy_Lcam6_1.cam.h1.0010-01-01-00000.nc',
            'key': iris.Constraint(cube_func=(lambda c: c.var_name == 'SHFLX'))
        }

    return(mydict)
####################

In [9]:
####################
## create correct time cubelist to save output data
cubelist_ct = iris.cube.CubeList()

##readin sfcwind
datasets=['slhf','DQ']  #['ERAI']  # ['u-bt405','u-bt406','u-bm213','u-be034','u-bd818']
for my_dataset in datasets:
    print('--> '+my_dataset)
    mydict = data_dict(my_dataset)
    datain = iris.load_cube(mydict['infile'],mydict['key'])
#    data_time= fx.correct_time(datain)

    cubelist_ct.append(datain)
#############################

--> slhf
--> DQ


/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/iris/fileformats/cf.py:584: UserWarning: Missing CF-netCDF formula term variable 'PS', referenced by netCDF variable 'lev'
  warnings.warn(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/iris/fileformats/cf.py:584: UserWarning: Missing CF-netCDF formula term variable 'PS', referenced by netCDF variable 'ilev'
  warnings.warn(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/iris/fileformats/cf.py:372: UserWarning: Missing CF-netCDF boundary variable 'time_bnds', referenced by netCDF variable 'time'
  warnings.warn(message % (name, nc_var_name))


In [10]:
#############################
##select region covers EURE4A
#############################
left = 0
right =360
bottom = -20
top = 20
y=fx.selregion(cubelist_ct[0],bottom,top,left,right)
x=fx.selregion(cubelist_ct[1],bottom,top,left,right)

# j/m2 to w/m2, upward is positive

y.data=y.data
x.data=x.data*1000

x.data[x.data==0]=np.nan
y.data[y.data==0]=np.nan
x.data = x.data.filled(fill_value=np.nan)
y.data = y.data.filled(fill_value=np.nan)




In [11]:
y

<iris 'Cube' of Surface latent heat flux / (W/m2) (time: 365; latitude: 42; longitude: 288)>

In [12]:
regyonx=fx.regress_yonx(x,y)


In [13]:
iris.save(regyonx,mydict['indir']+'/regress_slhf_DQ_LR_CAM6.nc')